# PYTHON FOR SUDOKU SOLVER

<li><a href="#un">1°) Règle du jeu</a></li>
<li><a href="#deux"> 2°) Mise en situation</a></li>
<li><a href="#trois">3°) Déterminer les blocs</a></li>
<li><a href="#quatre"> 4°) Déterminer les blocs</a></li>
<li><a href="#cinq"> 5°) Possibilités </a></li>
<li><a href="#six"> 6°) Algorithme de résolution </a></li>

### <a id="un">1°) Règles du jeu</a>

La règle du jeu du sudoku est très simple et tient en une phrase :
Remplir les cases vides avec les chiffres de 1 à 9, de telle sorte qu'ils n'apparaissent qu'une fois par ligne, par colonne et par carré de 3x3 cases.

Nul besoin d'être fort en math, la logique et le bon sens permettent de résoudre les grilles.

Une grille de sudoku est divisée en 9 lignes, 9 colonnes et 9 carrés.
- Chacun doit contenir tous les chiffres de 1 à 9.
- Chaque case reçoit un chiffre de 1 à 9. Elle fait partie de trois groupes à la fois : elle se trouve sur une ligne, une colonne et un carré en même temps.

Les groupes sont les éléments qui doivent contenir tous les chiffres de 1 à 9, ce sont :
- La ligne est un ensemble de neuf cases/cellules disposées horizontalement
- La colonne est un ensemble de neuf cases disposées verticalement
- Le carré est un ensemble de neuf cases disposées en carré de 3x3 cases, la grille étant composées de neuf de ces carrés.

### <a id="deux">2°) Mise en situation</a>

Pour commencer, importons les librairies nécessaires à la réalisation du programme.

In [2]:
import numpy as np ; import pandas as pd ; import random

 Présentons maintenant notre grille-témoin, un sudoku résolu qui nous permettra de vérifier la validité des codes à venir.

In [3]:
solved_example=np.array([[3, 9, 1,   2, 8, 6,   5, 7, 4],
                         [4, 8, 7,   3, 5, 9,   1, 2, 6],
                         [6, 5, 2,   7, 1, 4,   8, 3, 9],
              
                         [8, 7, 5,   4, 3, 1,   6, 9, 2],
                         [2, 1, 3,   9, 6, 7,   4, 8, 5],
                         [9, 6, 4,   5, 2, 8,   7, 1, 3],
              
                         [1, 4, 9,   6, 7, 3,   2, 5, 8],
                         [5, 3, 8,   1, 4, 2,   9, 6, 7],
                         [7, 2, 6,   8, 9, 5,   3, 4, 1]]) ; solved_example

array([[3, 9, 1, 2, 8, 6, 5, 7, 4],
       [4, 8, 7, 3, 5, 9, 1, 2, 6],
       [6, 5, 2, 7, 1, 4, 8, 3, 9],
       [8, 7, 5, 4, 3, 1, 6, 9, 2],
       [2, 1, 3, 9, 6, 7, 4, 8, 5],
       [9, 6, 4, 5, 2, 8, 7, 1, 3],
       [1, 4, 9, 6, 7, 3, 2, 5, 8],
       [5, 3, 8, 1, 4, 2, 9, 6, 7],
       [7, 2, 6, 8, 9, 5, 3, 4, 1]])

 Puis présentons la grille à compléter sur laquelle nous allons tester notre programme de résolution, où les cases à déterminer sont remplacées par des 0.

In [4]:
grille = np.array([[8, 0, 0,    0, 0, 0,    0, 0, 0],
                   [0, 0, 3,    6, 0, 0,    0, 0, 0],
                   [0, 7, 0,    0, 9, 0,    2, 0, 0],
                   
                   [0, 5, 0,    0, 0, 7,    0, 0, 0],
                   [0, 0, 0,    0, 4, 5,    7, 0, 0],
                   [0, 0, 0,    1, 0, 0,    0, 3, 0],
                   
                   [0, 0, 1,    0, 0, 0,    0, 6, 8],
                   [0, 0, 8,    5, 0, 0,    0, 1, 0],
                   [0, 9, 0,    0, 0, 0,    4, 0, 0]]) ; grille

array([[8, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 3, 6, 0, 0, 0, 0, 0],
       [0, 7, 0, 0, 9, 0, 2, 0, 0],
       [0, 5, 0, 0, 0, 7, 0, 0, 0],
       [0, 0, 0, 0, 4, 5, 7, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 3, 0],
       [0, 0, 1, 0, 0, 0, 0, 6, 8],
       [0, 0, 8, 5, 0, 0, 0, 1, 0],
       [0, 9, 0, 0, 0, 0, 4, 0, 0]])

 Si vous voulez le faire vous-même, libre à vous! Me concernant, j'y ai cumulé près de 5h à le résoudre (sans triche), et pourtant je suis un spécialiste! Donc autant vous dire que vous allez déguster...! Accessoirement, je vous affirme d'entrée de jeu que la solution est unique.

### <a id="trois">3°) Déterminer les blocs</a>

 Déterminer la ligne et la colonne d'un élément d'un sudoku, c'est facile! Pour obtenir le bloc auquel il appartient (un des 9 carrés de la grille), c'est plus compliqué! La fonction suivante s'en charge en prenant en paramètre un sudoku Z donné, ainsi que x et y, respectivement les numéros de ligne et de colonne (allant chacun de 0 à 8 dans l'encoding) de l'élément recherché.

In [5]:
def bloc(Z, x, y):
    if x in [0, 1, 2]: Z=Z[0:3, :]
    if x in [3, 4, 5]: Z=Z[3:6, :]
    if x in [6, 7, 8]: Z=Z[6:9, :]
    if y in [0, 1, 2]: Z=Z[:, 0:3]
    if y in [3, 4, 5]: Z=Z[:, 3:6]
    if y in [6, 7, 8]: Z=Z[:, 6:9]
    return(Z)

 Par exemple, repérez sur la grille à résoudre le 7 situé à la cinquième ligne et à la septième colonne. Il appartient au carré milieu droit de la grille. On peut le récupérer à partir de ses indices dans la grilles en prenant soin de décrémenter ces derniers pour tenir compte des conventions numpy.

In [6]:
bloc(grille, 4, 6)

array([[0, 0, 0],
       [7, 0, 0],
       [0, 3, 0]])

 Pour repérer les éléments dans un bloc, nous nous servirons plutôt de la fonction crushed_bloc qui prend les mêmes paramètres que la fonction précédente, mais qui nous renvoie le bloc sous forme de liste.

In [7]:
def crushed_bloc(Z, x, y):
    Z=bloc(Z, x, y)
    return(list(Z[0])+list(Z[1])+list(Z[2]))

crushed_bloc(grille, 4, 6)

[0, 0, 0, 7, 0, 0, 0, 3, 0]

### <a id="quatre">4°) Validation d'une grille</a>

 Pour commencer, créons une fonction qui valide la candidature d'un nombre, c'est-dire qui renverra False si ce nombre se trouve déjà ailleurs sur la même ligne, la même colonne ou le même bloc. 

In [8]:
def check_box(sudoku, x, y):
    VALIDITE=True
    
    nb=sudoku[x][y]
    Z=crushed_bloc(sudoku, x, y)
    
    if (list(sudoku[x]).count(nb)>=2) or (list(sudoku[:, y]).count(nb)>=2): VALIDITE=False
    if Z.count(nb)>=2: VALIDITE=False
    if nb==0: VALIDITE=False
    
    return(VALIDITE)

 Nous allons maintenant créer une fonction qui prend en argument une grille et qui renvoie True si le sudoku est complet, et sans erreur, False sinon.

In [9]:
def check_grid(sudoku):
    VALIDITE=True
        
    for x in range(9):
        for y in range(9):
            if check_box(sudoku, x, y)==False: VALIDITE=False
                
    return(VALIDITE)

In [10]:
check_grid(solved_example)

True

 On peut donc tester cette fonction sur notre grille-exemple ainsi que sur notre grille à remplir.

In [11]:
print("solved_example: ", check_grid(solved_example))
print("grille: ", check_grid(grille))

solved_example:  True
grille:  False


### <a id="cinq">5°) Les possibilités</a>

 Un sudoku digne de ce nom possède une unique solution. À moins que vous découvriez ce jeu en lisant ce script, vous savez d'ores et déjà qu'on a tendance à commencer un sudoku en écrivant en tout petit les possibilités en haut à gauche dans les cases. Par exemple, s'il on se réfère à la case vide de notre grille à la 1ère ligne et à la deuxième colonne ([0, 1] dans le code), on relève rapidement que les nombres candidats sont: 1, 2, 4 et 6.
    
La fonction suivante va nous permettre, pour une case vide donnée, de relever la listes des candidats potentiels de la grille.

In [12]:
def candidats(sudoku, x, y):
    
    P = []
    for i in range(1, 10):
        sudoku[x][y]=i
        if check_box(sudoku, x, y):
            P.append(i)
        sudoku[x][y]=0
    return(P)


candidats(grille, 0, 1)

[1, 2, 4, 6]

 Nous pourrons donc établir la liste des possibilités pour chaque case 0 d'un sudoku à remplir.

### <a id="six">6°) Algorithme de résolution</a>

 Nous sommes maintenant en mesure de créer l'algoritme de résolution de sudoku. Pour cela, nous allons utiliser une fonction récursive qui va tester l'ensemble des possibilités, pour ne nous retourner que la solution exacte du sudoku que nous entrons en paramètre.

In [13]:
def solver(sudoku):
    for x in range(9):
        for y in range(9):
            if sudoku[x][y]==0:
                candidates = candidats(sudoku, x, y)
                for p in candidates:
                    sudoku[x][y]=p
                    solver(sudoku)
                    sudoku[x][y]=0
                return   
    print(sudoku)

 Enfin, nous pouvons appliquer notre algorithme sur la grille que nous avons entrée plus haut et afficher la solution du sudoku.

In [14]:
solver(grille)

[[8 1 2 7 5 3 6 4 9]
 [9 4 3 6 8 2 1 7 5]
 [6 7 5 4 9 1 2 8 3]
 [1 5 4 2 3 7 8 9 6]
 [3 6 9 8 4 5 7 2 1]
 [2 8 7 1 6 9 5 3 4]
 [5 2 1 9 7 4 3 6 8]
 [4 3 8 5 2 6 9 1 7]
 [7 9 6 3 1 8 4 5 2]]


 Je vous laisse vérifier que la grille ne contient aucune erreur. Bien entendu, cette solution est unique, auquel cas notre algorithme nous aurait afficher l'ensemble des solutions. Inutile de vous parler du temps d'exécution et de l'affichage si l'on avait rentré une grille nulle à résoudre. Si l'on devait attendre que l'ordinateur calcule et affiche les <B>201 105 135 151 764 480</B> possibilités de sudokus réalisables, Ulysse aurait eu le temps de terminer son voyage!